In [1]:
import pandas as pd
from pandas import read_csv
from matplotlib import pyplot as plt
import numpy as np
import chess
from chess import pgn
import os

In [2]:
master_data = True
reuse_master_data = False # change after initial data processign
if not master_data:
    df = read_csv('../data/chess.csv') # adjust to use master_data.pgn
    df = df[df['winner']!='draw']
    moves = df['moves'].values[:100]
    # print (moves)
    winner = df['winner'].values

In [3]:
def matrix_rep(board): 
    board_epd = board.epd()
    mat = []  
    pieces = board_epd.split(" ", 1)[0]
    rows = pieces.split("/")
    for r in rows:
        sub_mat = []  
        for pic in r:
            if pic.isdigit():
                for i in range(0, int(pic)):
                    sub_mat.append('.')
            else:
                sub_mat.append(pic)
        mat.append(sub_mat)
    return mat

In [4]:
def trans_code(matrix,pieces_dict):
    rows = []
    for row in matrix:
        terms = []
        for term in row:
            terms.append(pieces_dict[term])
        rows.append(terms)
    return rows

one_hot_pieces_dict = {
    'p' : [1,0,0,0,0,0,0,0,0,0,0,0],
    'P' : [0,0,0,0,0,0,1,0,0,0,0,0],
    'n' : [0,1,0,0,0,0,0,0,0,0,0,0],
    'N' : [0,0,0,0,0,0,0,1,0,0,0,0],
    'b' : [0,0,1,0,0,0,0,0,0,0,0,0],
    'B' : [0,0,0,0,0,0,0,0,1,0,0,0],
    'r' : [0,0,0,1,0,0,0,0,0,0,0,0],
    'R' : [0,0,0,0,0,0,0,0,0,1,0,0],
    'q' : [0,0,0,0,1,0,0,0,0,0,0,0],
    'Q' : [0,0,0,0,0,0,0,0,0,0,1,0],
    'k' : [0,0,0,0,0,1,0,0,0,0,0,0],
    'K' : [0,0,0,0,0,0,0,0,0,0,0,1],
    '.' : [0,0,0,0,0,0,0,0,0,0,0,0],
}

In [5]:
if not master_data:
    X = []
    y = []
    for game in moves:
        index = list(moves).index(game)
        all_moves = game.split()
        total_moves = len(all_moves)
        # print (index)
        if winner[index] == 'black':
            game_winner = -1
        else:
            game_winner = 1
        board = chess.Board()
        for i in range(len(all_moves)):
            board.push_san(all_moves[i])
            value = game_winner * (i/total_moves)
            matrix = matrix_rep(board.copy())
            rows = trans_code(matrix,one_hot_pieces_dict)
            X.append([rows])
            y.append(value)
    X = np.array(X).reshape(len(X),8,8,12)
    y = np.array(y)
    X.shape
else:
    if (not reuse_master_data):
        pgn_data = open("../data/twic1356.pgn")
        X = []
        y = []
        num_game = 0
        while (not pgn_data.closed):
            try:
                game = chess.pgn.read_game(pgn_data)
                result = game.headers["Result"][2]
                game_winner = 0
                if result == '0':
                    game_winner = 1 # white win
                if result == '1':
                    game_winner = -1 # black win
                board = chess.Board()
                all_moves = game.mainline_moves()
                total_moves = 150 # change ?
                i = 0
                for move in all_moves:
                    board.push(move)
                    value = game_winner * (i/total_moves) # maybe change to just value = game_winner
                    matrix = matrix_rep(board.copy())
                    rows = trans_code(matrix, one_hot_pieces_dict)
                    X.append([rows])
                    y.append(value)
                    i += 1
            except:
                break
            num_game += 1
            print (num_game)
            # print (np.array(X).shape)
        X = np.array(X).reshape(len(X),8,8,12)
        y = np.array(y)
        # X.shape
        np.save('inputs.npy', X)
        np.save('outputs.npy', y)

    else:
        X = load('inputs.npy')
        y = load('outputs.npy')

FileNotFoundError: [Errno 2] No such file or directory: '../data/twic1356.pgn'